# 4. Kubernetes First Deployment

Now it's time to deploy the sample application to the previously created GKE cluster.

In [4]:
export PROJECT_ID=k8s-hands-on-quality
export CLUSTER_NAME=k8s-hands-on
export CLUSTER_ZONE=us-west1-c

## 4.1 Prepare Image

Push built sample applications's image to GCR.

In [5]:
docker tag k8s-hands-on:v1 gcr.io/$PROJECT_ID/k8s-hands-on:v1

In [7]:
gcloud docker -- push gcr.io/$PROJECT_ID/k8s-hands-on:v1


As an alternative, use `gcloud auth configure-docker` to configure `docker` to
use `gcloud` as a credential helper, then use `docker` as you would for non-GCR
registries, e.g. `docker pull gcr.io/project-id/my-image`. Add
`--verbosity=error` to silence this warning: `gcloud docker
--verbosity=error -- pull gcr.io/project-id/my-image`.

See: https://cloud.google.com/container-registry/docs/support/deprecation-notices#gcloud-docker

The push refers to repository [gcr.io/k8s-hands-on-quality/k8s-hands-on]

8414bb0b: Preparing 
fd394570: Preparing 
v1: digest: sha256:6fab595207df9755d07c80fb0a7c29a02ac28ae37f1cbf52ab7e1607ab99d7d7 size: 949


## 4.2 Deploy Container

### 4.2.1 Create a namespace

In [9]:
kubectl create namespace k8s-hands-on

namespace/k8s-hands-on created



### 4.2.2 Create a pod deployment

In [24]:
cat k8s/first_deployment/deployment.yaml

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  labels:
    name: k8s-first-app
  name: k8s-first-app
  namespace: k8s-hands-on
spec:
  replicas: 1
  template:
    metadata:
      labels:
        name: k8s-first-app
    spec:
      containers:
      - name: app
        image: gcr.io/YOUR_PROJECT_ID/k8s-hands-on:v1
        ports:
          - name: app
            containerPort: 8888


Use sed to replace the GCP Project ID,

In [28]:
sed -e "s/YOUR_PROJECT_ID/${PROJECT_ID}/g" -i k8s/first_deployment/deployment.yaml

In [29]:
cat k8s/first_deployment/deployment.yaml

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  labels:
    name: k8s-first-app
  name: k8s-first-app
  namespace: k8s-hands-on
spec:
  replicas: 1
  template:
    metadata:
      labels:
        name: k8s-first-app
    spec:
      containers:
      - name: app
        image: gcr.io/k8s-hands-on-quality/k8s-hands-on:v1
        ports:
          - name: app
            containerPort: 8888


Apply this to the cluster

In [30]:
kubectl apply -f k8s/first_deployment/deployment.yaml

deployment.extensions/k8s-first-app created


In [32]:
kubectl get deployments -n k8s-hands-on

NAME            DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
k8s-first-app   1         1         1            1           25s


In [33]:
kubectl get pods -n k8s-hands-on

NAME                             READY     STATUS    RESTARTS   AGE
k8s-first-app-79fbbbc766-pc54f   1/1       Running   0          35s


## 4.3 Create Service

In [34]:
cat k8s/first_deployment/service.yaml

apiVersion: v1
kind: Service
metadata:
  labels:
    name: k8s-first-service
  name: k8s-first-service
  namespace: k8s-hands-on
spec:
  type: NodePort
  ports:
  - port: 80
    targetPort: 8888
  selector:
    name: k8s-first-app


In [35]:
kubectl apply -f k8s/first_deployment/service.yaml

service/k8s-first-service created


In [36]:
kubectl get svc -n k8s-hands-on

NAME                TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
k8s-first-service   NodePort   10.31.248.239   <none>        80:30133/TCP   10s


## 4.4 Create Ingress to expose the service

In [37]:
cat k8s/first_deployment/ingress.yaml

apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  name: k8s-first-ingress
  namespace: k8s-hands-on
  annotations:
    kubernetes.io/tls-acme: "true"
    kubernetes.io/ingress.class: "gce"
    kubernetes.io/ingress.allow-http: "true"
spec:
  backend:
    serviceName: k8s-first-service
    servicePort: 80


In [38]:
kubectl apply -f k8s/first_deployment/ingress.yaml

ingress.extensions/k8s-first-ingress created


In [39]:
kubectl get ingress -n k8s-hands-on

NAME                HOSTS     ADDRESS         PORTS     AGE
k8s-first-ingress   *         35.241.16.212   80        1m


Wait a few seconds and you will be able to access the content

In [40]:
curl http://35.241.16.212/

Hello!

## 4.5 Clean up

In [41]:
kubectl delete ingress/k8s-first-ingress -n k8s-hands-on

ingress.extensions "k8s-first-ingress" deleted


In [42]:
kubectl delete service/k8s-first-service -n k8s-hands-on

service "k8s-first-service" deleted


In [43]:
kubectl delete deployment/k8s-first-app -n k8s-hands-on

deployment.extensions "k8s-first-app" deleted


In [44]:
kubectl get pods -n k8s-hands-on

No resources found.


In [45]:
kubectl get svc -n k8s-hands-on

No resources found.


In [46]:
kubectl get ingress -n k8s-hands-on

No resources found.


In [47]:
kubectl delete namespace k8s-hands-on

namespace "k8s-hands-on" deleted
